In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# 요청하신 종목명과 종목코드 딕셔너리
stocks = {
    "한화오션": "042660",
    "HD한국조선해양": "009540",
    "HD현대마린솔루션": "011200",
    "HD현대마린엔진": "077970",
    "HD현대중공업": "329180",
    "현대힘스": "214680",
    "삼성중공업": "010140",
    "SK오션플랜트": "095190",
    "STX엔진": "077970",
    "한화엔진": "045100",
    "일승": "333430",
    "오리엔탈정공": "014940",
    "케이에스피": "073010",
    "한국카본": "017960",
    "동성화인텍": "033500",
    "대창솔루션": "096350",
    "엔케이": "085310",
    "화인베스틸": "133820",
    "HJ중공업": "097230",
    "세진중공업": "075580",
    "케이프": "064820",
    "대양전기공업": "108380",
    "동방선기": "099410",
    "한라IMS": "092460",
    "인화정공": "101930",
    "디케이락": "105740",
    "태광": "023160",
    "성광벤드": "014620",
    "비엠티": "086670",
    "하이록코리아": "013030"
}

# 요청 헤더 설정
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'
}

# 데이터 저장용 딕셔너리 (날짜를 키로 사용)
data_dict = {}

# 모든 종목 데이터 수집
for stock_name, stock_code in stocks.items():
    for page in range(1, 6):  # 1~5페이지 조회
        url = f"https://finance.naver.com/item/frgn.naver?code={stock_code}&page={page}"
        response = requests.get(url, headers=headers)

        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            rows = soup.select("table.type2 tr")

            for row in rows:
                cols = row.select("td")  # 테이블의 각 열(td) 추출
                
                if len(cols) >= 9:  # 데이터가 존재하는 행인지 확인
                    date = cols[0].text.strip()  # 날짜
                    change_rate = cols[3].text.strip().replace('%', '')  # 변동률

                    if date and change_rate:  # 유효한 데이터만 추가
                        if date not in data_dict:
                            data_dict[date] = {}
                        
                        data_dict[date][f"{stock_name}"] = change_rate
        else:
            print(f"{stock_name}의 페이지 {page}를 불러오지 못했습니다. 상태 코드:", response.status_code)

# 딕셔너리를 데이터프레임으로 변환
df = pd.DataFrame.from_dict(data_dict, orient='index')
df.index.name = '날짜'

# 날짜 순 정렬
df = df.sort_index(ascending=True)

# 엑셀로 저장
file_name = "조선업.xlsx"
df.to_excel(file_name)

print(f"모든 종목의 변동률 데이터가 '{file_name}'로 저장되었습니다.")


모든 종목의 변동률 데이터가 '조선업.xlsx'로 저장되었습니다.
